In [33]:
# LSTM for international airline passengers problem with regression framing
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [34]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=8):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [35]:
# fix random seed for reproducibility
numpy.random.seed(8)

In [36]:

# load the dataset
dataframe = read_csv('wavelet_kesinga.csv', engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')


In [37]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 8))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [38]:
# reshape into X=t and Y=t+1
look_back = 8
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [41]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [42]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)



# invert predictions
# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])

Epoch 1/100
319/319 - 5s - loss: 6.3596 - 5s/epoch - 17ms/step
Epoch 2/100
319/319 - 1s - loss: 2.1461 - 1s/epoch - 4ms/step
Epoch 3/100
319/319 - 1s - loss: 1.5872 - 1s/epoch - 4ms/step
Epoch 4/100
319/319 - 1s - loss: 1.3431 - 1s/epoch - 5ms/step
Epoch 5/100
319/319 - 2s - loss: 1.1552 - 2s/epoch - 5ms/step
Epoch 6/100
319/319 - 2s - loss: 1.0028 - 2s/epoch - 5ms/step
Epoch 7/100
319/319 - 1s - loss: 0.9117 - 1s/epoch - 4ms/step
Epoch 8/100
319/319 - 1s - loss: 0.8443 - 1s/epoch - 4ms/step
Epoch 9/100
319/319 - 1s - loss: 0.8083 - 1s/epoch - 4ms/step
Epoch 10/100
319/319 - 1s - loss: 0.7794 - 1s/epoch - 4ms/step
Epoch 11/100
319/319 - 1s - loss: 0.7603 - 1s/epoch - 4ms/step
Epoch 12/100
319/319 - 1s - loss: 0.7369 - 1s/epoch - 4ms/step
Epoch 13/100
319/319 - 1s - loss: 0.7139 - 1s/epoch - 4ms/step
Epoch 14/100
319/319 - 1s - loss: 0.7067 - 1s/epoch - 4ms/step
Epoch 15/100
319/319 - 1s - loss: 0.6916 - 1s/epoch - 4ms/step
Epoch 16/100
319/319 - 1s - loss: 0.6836 - 1s/epoch - 5ms/step


In [43]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

5/5 [==============================] - 0s 6ms/step


In [44]:
testPredict

array([[4.318134 ],
       [3.9176176],
       [3.5164092],
       [3.3517554],
       [3.000716 ],
       [4.592175 ],
       [3.8134205],
       [3.8453767],
       [5.186933 ],
       [6.234332 ],
       [5.8558536],
       [3.987165 ],
       [3.0738902],
       [2.9441633],
       [2.5966296],
       [2.3017743],
       [3.4995239],
       [3.4877617],
       [3.4474926],
       [4.0345206],
       [4.017188 ],
       [3.307995 ],
       [3.7382064],
       [3.6904943],
       [3.942324 ],
       [3.3781483],
       [4.3416624],
       [3.751744 ],
       [3.411881 ],
       [3.0633202],
       [5.073269 ],
       [5.166176 ],
       [4.358286 ],
       [5.31682  ],
       [5.7233324],
       [5.1492763],
       [4.3343782],
       [3.7283194],
       [3.5612245],
       [2.9624307],
       [2.6283398],
       [3.72946  ],
       [4.2775245],
       [4.614822 ],
       [3.8043482],
       [4.4610343],
       [4.5770626],
       [4.55642  ],
       [3.2934105],
       [2.502069 ],


In [45]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY.reshape(-1, 8)])
testPredict = scaler.inverse_transform(testPredict.reshape(-1, 8))
testY = scaler.inverse_transform([testY.reshape(-1, 8)])

ValueError: non-broadcastable output operand with shape (319,1) doesn't match the broadcast shape (319,8)

In [ ]:
# trainPredict = scaler.inverse_transform(trainPredict.reshape(-1, 1))
# trainY = scaler.inverse_transform([trainY.reshape(-1, 1)])
testPredict = scaler.inverse_transform(testPredict.reshape(-1, 1))
testY = scaler.inverse_transform([testY.reshape(-1, 1)])


In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [ ]:
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_evaluation_metrics(y_observed, y_predicted):
     # Convert to NumPy arrays if they are lists
    if isinstance(y_observed, list):
        y_observed = np.array(y_observed)
    if isinstance(y_predicted, list):
        y_predicted = np.array(y_predicted)
        
    metrics = {}

    # Calculate ME (Mean Error)
    metrics['ME'] = np.mean(y_observed - y_predicted)

    # Calculate MAE (Mean Absolute Error)
    metrics['MAE'] = mean_absolute_error(y_observed, y_predicted)

    # Calculate MSE (Mean Squared Error)
    metrics['MSE'] = mean_squared_error(y_observed, y_predicted)

    # Calculate RMSE (Root Mean Squared Error)
    metrics['RMSE'] = np.sqrt(metrics['MSE'])

    # Calculate NRMSE (Normalized Root Mean Squared Error)
    metrics['NRMSE'] = metrics['RMSE'] / (np.max(y_observed) - np.min(y_observed))

    # Calculate PBIAS (Percent Bias)
    metrics['PBIAS'] = np.mean(100 * (y_observed - y_predicted) / np.mean(y_observed))

    # Calculate RSR (Root Mean Square Ratio)
    metrics['RSR'] = metrics['RMSE'] / np.std(y_observed)

    # Calculate rSD (Ratio of the Standard Deviation)
    metrics['rSD'] = np.std(y_observed - y_predicted) / np.std(y_observed)

    # Calculate NSE (Nash-Sutcliffe Efficiency)
    metrics['NSE'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((y_observed - np.mean(y_observed)) ** 2))

    # Calculate mNSE (Modified Nash-Sutcliffe Efficiency)
    metrics['mNSE'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((y_observed - np.mean(y_observed)) ** 2))

    # Calculate rNSE (Relative Nash-Sutcliffe Efficiency)
    metrics['rNSE'] = metrics['NSE'] / np.var(y_observed)

    # Calculate d (Index of Agreement)
    metrics['d'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((np.abs(y_predicted - np.mean(y_observed)) +
                                                                       np.abs(y_observed - np.mean(y_observed))) ** 2))

    # Calculate md (Modified d Index)
    metrics['md'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((np.abs(y_predicted - np.mean(y_observed)) +
                                                                         np.abs(y_observed - np.mean(y_observed))) ** 2))

    # Calculate rd (Relative d Index)
    metrics['rd'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((np.abs(y_predicted - np.mean(y_observed)) +
                                                                         np.abs(y_observed - np.mean(y_observed))) ** 2))

    # Calculate cp (Coefficient of Performance)
    metrics['cp'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((np.abs(y_predicted - y_observed.mean()) +
                                                                         np.abs(y_observed - y_observed.mean())) ** 2))

    # Calculate r (Index of Agreement)
    metrics['r'] = 1 - (np.sum((y_observed - y_predicted) ** 2) / np.sum((np.abs(y_predicted - y_observed.mean()) +
                                                                        np.abs(y_observed - y_observed.mean())) ** 2))

    # Calculate R2 (Coefficient of Determination)
    metrics['R2'] = r2_score(y_observed, y_predicted)

    return metrics

In [ ]:
evaluation_metrics = calculate_evaluation_metrics(trainY, trainPredict)
print(evaluation_metrics)